In [ ]:
from google.colab import drive
# Mount google drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ML projects
%ls

Mounted at /content/drive
/content/drive/MyDrive/ML projects
COVID/


In [ ]:
import pandas as pd

# Load dataset file
df = pd.read_csv("confirmed_new.csv")
df.head()

FileNotFoundError: ignored

In [ ]:
df2 = df.drop(df.columns[[0, 1]], axis=1)
df2.head()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

df_standard_scaled = df2.copy()
df_min_max_scaled = df2.copy()

# Standardization  
df_standard_scaled.iloc[:,1:] = pd.DataFrame(StandardScaler().fit_transform(df2.loc[:, df2.columns != "Country/Region"]))
# Normalization or Min-Max Scaling
df_min_max_scaled.iloc[:,1:] = pd.DataFrame(MinMaxScaler().fit_transform(df2.loc[:, df2.columns != "Country/Region"]))

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 10, max_iter=600)

y_df2 = kmeans.fit_predict(df2.iloc[:,[1,2,3,4]])
y_df_standard_scaled = kmeans.fit_predict(df2.iloc[:,[1,2,3,4]])
y_df_min_max_scaled = kmeans.fit_predict(df2.iloc[:,[1,2,3,4]])

df2['Cluster'] = y_df2
df_standard_scaled['Cluster'] = y_df_standard_scaled
df_min_max_scaled['Cluster'] = y_df_min_max_scaled

In [ ]:
df_standard_scaled['monthly_cases_mean'] = df2[["March","April","May","June"]].mean(axis=1)
df_standard_scaled['monthly_cases_std'] = df2[["March","April","May","June"]].std(axis=1)
df_standard_scaled.head()

In [ ]:
df_min_max_scaled['monthly_cases_mean'] = df_min_max_scaled[["March","April","May","June"]].mean(axis=1)
df_min_max_scaled['monthly_cases_std'] = df_min_max_scaled[["March","April","May","June"]].std(axis=1)
df_min_max_scaled.head()

In [ ]:
df2['monthly_cases_mean'] = df2[["March","April","May","June"]].mean(axis=1)
df2['monthly_cases_std'] = df2[["March","April","May","June"]].std(axis=1)
df2.head()

In [ ]:
df2_g = df2.groupby(['Cluster'])
df_min_max_scaled_g = df_min_max_scaled.groupby(['Cluster'])
df_standard_scaled_g = df_standard_scaled.groupby(['Cluster'])

In [ ]:
df2_avg_monthly_cases_std = {}
df_min_max_avg_monthly_cases_std = {}
df_standard_avg_monthly_cases_std = {}

for i in range(10):
  df2_avg_monthly_cases_std[i] = int(df2_g.get_group(i)[['monthly_cases_std']].mean()), int(df2_g.get_group(i)["March"].count())
  df_min_max_avg_monthly_cases_std[i] = float(df_min_max_scaled_g.get_group(i)[['monthly_cases_std']].mean()), int(df_min_max_scaled_g.get_group(i)["March"].count())
  df_standard_avg_monthly_cases_std[i] = float(df_standard_scaled_g.get_group(i)[['monthly_cases_std']].mean()), int(df_standard_scaled_g.get_group(i)["March"].count())

In [ ]:
final_df = pd.DataFrame.from_dict(df2_avg_monthly_cases_std, orient ='index')
final_df_min_max = pd.DataFrame.from_dict(df_min_max_avg_monthly_cases_std, orient ='index')
final_df_standard = pd.DataFrame.from_dict(df_standard_avg_monthly_cases_std, orient ='index')

In [ ]:
final_df.rename(columns={0: 'Group Average STD of Monthly Confirmed Cases', 1: 'Number of Group Members'}, inplace=True)
final_df.sort_values(by=['Number of Group Members'], ascending=False)

In [ ]:
final_df_standard.rename(columns={0: 'Group Average STD of Monthly Confirmed Cases', 1: 'Number of Group Members'}, inplace=True)
final_df_standard.sort_values(by=['Number of Group Members'], ascending=False)

In [ ]:
final_df_min_max.rename(columns={0: 'Group Average STD of Monthly Confirmed Cases', 1: 'Number of Group Members'}, inplace=True)
final_df_min_max.sort_values(by=['Number of Group Members'], ascending=False)

In [ ]:
for i in range(10):
  if((df2_avg_monthly_cases_std[i][1]) < 10):
    print("Without normalization or standardization, cluster ", i ,"has ", df2_avg_monthly_cases_std[i][1] , "cases which are ", list(df2_g.get_group(i)["Country/Region"]))

print("**********************************************************************************************************************\n")
for i in range(10):
  if((df_min_max_avg_monthly_cases_std[i][1]) < 10):
    print("With Min Max scaling, cluster ", i ,"has ", df_min_max_avg_monthly_cases_std[i][1] , "cases which are ", list(df_min_max_scaled_g.get_group(i)["Country/Region"]))

print("**********************************************************************************************************************\n")
for i in range(10):
  if((df_standard_avg_monthly_cases_std[i][1]) < 10):
    print("With standard scaling, cluster ", i ,"has ", df_standard_avg_monthly_cases_std[i][1] , "cases which are ", list(df_standard_scaled_g.get_group(i)["Country/Region"]))